# (some) Related Work 
- A spectral Monte Carlo method for the Poisson equation
- Chebyshev Greeks Smoothing Gamma without Bias
- Approximation Theory and Approximation Practice (Lloyd N. Trefethen)
- Regression-based Monte Carlo Integration
- A new sequential algorithm for L2-approximation and application to Monte-Carlo integration (SALT)

We haven't looked into the literature for random ODEs (random start and coefficients not stochastic differential equations)



# Abstract
(we will write this later)

# Russian Roulette Modded
In period1 we missed a simple insight of Russian Roulette. Instead of killing of simulations completely you can substitute an approximation instead again without changing the expectance. This shouldn't be confused with control variates although both need approximations of the solution. <br>

To get our example in the code to work we did some algebra to figure out the modification:

In [143]:
from random import random
from math import exp

def approx_sol(t):
    return 1+t    

def Y(t):
    k = t if t<1 else 1
    return (1+t*Y(random()*t)-(1-k)*approx_sol(t))/k if random()<k else approx_sol(t)

sol = 0
nsim = 10**3
t = 3

for _ in range(nsim):
    sol += Y(t)/nsim

percentage_error = (sol - exp(t))/exp(t)

print(f"exp({t}) is approx = {sol},",f"%error = {percentage_error}")

exp(3) is approx = 20.266231000573733, %error = 0.008996228384488094


# RMC exp Dy
In period1 we weren't sure we could also recurse on derivate of a the same order of the underlying differential equation. We show that it is possible with an example:

$$
y'= y+1, y(0)=1
$$
if we have a random variable $Y(t)$ such that $E[Y(t)]=y(t)$ from this we construct a random variable $DY(t)$ such that $E[DY(t)]= y'(t)$ in this case $DY(t) = Y(t) + 1$. <br>

Let's use this to make a recursive Monte Carlo scheme that also recurses on the derivative:
$$
\begin{align*}
y'      &= \frac{1}{2}y + \frac{1}{2}y' + \frac{1}{2} \Leftrightarrow \\
y (t)   &=  y(0) + \int_{0}^{t} \frac{1}{2}y(s) + \frac{1}{2}y'(s) + \frac{1}{2} ds\\
        &=  1 + t E \left[\frac{1}{2}y(U) + \frac{1}{2}y'(U) + \frac{1}{2} \right] \\
        &=  E \left[1 +  \frac{t}{2}Y(U) + \frac{t}{2}DY(U) + \frac{t}{2} \right] \\
\end{align*}
$$
This give following recursion relation for random variables:
$$
\begin{align*}
DY(t) &= Y(t) + 1 \\     
Y(t) &= 1+0.5tY(U) + 0.5tDY(U) + 0.5 t        
\end{align*}
$$
with $U = \text{Uniform}(0,t)$. We modified this with Russian Roulette in the implementation to make it non branching.

In [139]:
from random import random
from math import exp

#let t<1
def Y(t):
    if random()>t:
        return 1+0.5*t 
    s = random()*t
    return 1+ Y(s)  + 0.5*t if random()<0.5 else 1 + DY(s) + 0.5*t

def DY(t):
    return Y(t)+1
    
sol = 0
nsim = 10**3
t = 0.8 # <1

for _ in range(nsim):
    sol += Y(t)/nsim

s = 2*exp(t)-1 #analytic solution
percentage_error = (sol - s)/s

print(f"sol({t}) is approx {sol},",f"%error = {percentage_error}")

sol(0.8) is approx 3.3887756318060505, %error = -0.01805411397379012


# Random ODEs
Let's get a quick example in (and maybe work out more later):
$$
y' = Ay, y(0)=1 , A = \text{Uniform}(0,1) 
$$
With solution $y(t)= e^{tA}$ which is a random variable. With the techniques we have right now we can simulate unbiased estimates of simulations of the solution (yeah randomness on randomness).With these you can get "weak" solutions ($E_y[f(y(t))]$ with $f$ analytic) to the problem: 
$$
\begin{align*}
E_y[f(y(t))] &= E_y[E_A[f(y(t)) \mid  A]] \\    
           &= E_y[E_A[f(y(t,A))]] \\    
            &= E_y[E_A[f(E_Y[Y(t,A)])]] \\
\end{align*}
$$
remember how we dealt with $f(E_Y[Y(t,A)])$ in period1? This makes things like unbiased estimators of the expectance and variance possible which can be used to get confidence intervals using Chebychev's inequality (https://en.wikipedia.org/wiki/Chebyshev%27s_inequality). <br>

In our example we can find the following by hand:
$$
\begin{align*}
E_A[y(t)] &=  \frac{e^{t}}{t} - \frac{1}{t} \\ 
E_A[y^{2}(t)] &=  \frac{e^{2t}}{2t} - \frac{1}{2t}  
\end{align*}
$$
In code all this looks like this:

In [105]:
from random import random
from math import exp,sqrt

#estimator for y(t,a)
def Ya(t,a):
    return (1+a*Ya(random()*t,a) if random()<t else 1) if t<1 else 1+t*a*Ya(random()*t,a)

# estimator for y(t)
def Y(t):
    return Ya(t,random())
    
# estimator for y(t)**2
def Y2(t):
    A = random()
    return Ya(t,A)*Ya(t,A) 

sol = 0
sol2 = 0
nsim = 10**4
t = 3 # 0<t

for _ in range(nsim):
    sol += Y(t)/nsim
    sol2 += Y2(t)/nsim

s =  exp(t)/t -1/t #analytic solution
percentage_error = (sol - s)/s

s2 =  exp(2*t)/(2*t) -1/(2*t) #analytic solution
percentage_error2 = (sol2 - s2)/s2

print(f"E(y({t}))   is approx {sol},",f"%error = {percentage_error}")
print(f"E(y^2({t})) is approx {sol2},",f"%error = {percentage_error2}")

sd = sqrt(sol2-sol**2)
print(f"standard deviation of y(3) is approx (biased) {sd}")

E(y(3))   is approx 6.459277460154641, %error = 0.015315024065219401
E(y^2(3)) is approx 62.83166864932534, %error = -0.06321312492576975
standard deviation of y(3) is approx (biased) 4.594497071722166


# From ODEs to Integral Equations (continuation)
We continue our discussion on how to turn ODEs into integral equations. Let's start from following form:
$$
L(y)= f 
$$ 
where $L$ is a linear operator, $f$ be generic for now and some initial/boundary condition for $y$. All the methods we have in mind are some kind of integral transform:
$$
y(t) = \int_{\Omega} \varphi(x) K(t,x)dx
$$
where the integral may also be a summation and we may chose $K(t,x), \Omega$ and let $\varphi(x)$ be our new unknown. Note that we don't know that such representation exist or even is well defined but we continue and fix this in the future (hopefully). If you sub this into the first equation you obtain the following:

$$
\begin{align*}
L \left(\int_{\Omega} \varphi(x) K(t,x)dx \right) &= f \Leftrightarrow \\
\int_{\Omega} \varphi(x) L \left(K(t,x) \right)dx &= f 
\end{align*}
$$ 

this is a Fredholm integral equation of the first type in $\varphi(x)$ (https://en.wikipedia.org/wiki/Fredholm_integral_equation) if we had let $\Omega$ depend on $t$ in a certain way we would have obtained a Voltera integral equation of the second kind (https://en.wikipedia.org/wiki/Volterra_integral_equation). <br> 

## Source Green Function 
To make this a little more concrete we derive the method of source green functions in this framework (https://en.wikipedia.org/wiki/Green%27s_function). This can be done by choosing the free things in a way something nice happens: 
$$
L \left(K(t,x) \right) = \delta(t-x)
$$

with the same  domain and initial/boundary conditions such that the ones of $y$ hold. Choosing the initial/boundary conditions that way is easy but not trivial. <br> 

For a dirichlet boundary condition in $s$ the following can be done:
$$
\begin{align*}
y(s) &= \int_{\Omega} \varphi(x) K(s,x)dx \Leftarrow \\
K(s,x) &= \frac{y(s) l(x)}{\varphi(x)} \text{ and } 1 = \int_{\Omega}l(x)dx  
\end{align*}
$$
For a Neumann boundary condition in $s$ the following can be done:

$$
\begin{align*}
y'(s) &= \int_{\Omega} \varphi(x) K'(s,x)dx \Leftarrow \\
K'(s,x) &= \frac{y'(s) l(x)}{\varphi(x)} \text{ and } 1 = \int_{\Omega}l(x)dx  
\end{align*}
$$

with $l$ arbitrary. Linear type of initial/boundary conditions are very similar to this. The $\varphi = f$ in the initial/boundary condition is annoying but can be avoided by making the original initial/boundary condition $0$ by splitting explained in period1. <br>

Going back because of our choice the following thing can be derived

$$
\begin{align*}
\int_{\Omega} \varphi(x) L \left(K(t,x) \right)dx &= f \Leftrightarrow \\
\int_{\Omega} \varphi(x) \delta(t-x)dx &= f \Leftrightarrow \\
\varphi(t) &= f 
\end{align*}
$$ 

## Boundary Green Function
The previous deals with the source term in the split of period1. Let's try to deal with the boundary term in a similar way. Make $\Omega$ the boundary and choose $K(t,x)$ such that the initial/boundary conditions hold. 

$$
y(t)= \int_{\partial B} \varphi(x) K(t,x) dx
$$

Trying to enforce the same property as last time doesn't work
$$
\begin{align*}
    
L(y(t)) &=\int_{\partial B} \varphi(x) L(K(t,x)) dx = 0 \Leftrightarrow \\
L(y(t)) &=\int_{\partial B} \varphi(x) \delta(t-x) dx =0 \Leftrightarrow \\
\varphi(t)&= 0  \Rightarrow \\
y(t) &= 0
\end{align*}
$$
This only works if you have zero initial/boundary conditions ... otherwise you can't chose $K(t,x)$ such that the boundary conditions hold. You need to do something more to make this work:

For a dirichlet boundary condition in $s$ the following can be done:
$$
\begin{align*}
    L(y) &= 0 \Leftrightarrow \\
    L(y)+y &= y \Leftrightarrow \\
    \tilde{L}(y) &= y 
\end{align*}
$$
The reason why we put a $y$ on the right side is because we know it on the boundary later down the derivation it changes $\varphi$.

$$
\begin{align*}
    \tilde{L}(y(t)) &= \int_{\partial B} \varphi(x) \tilde{L}(K(t,x)) dx =y(t) \Leftrightarrow \\ 
    \tilde{L}(y(t)) &= \int_{\partial B} \varphi(x)  \delta(t-x)dx =y(t) \Leftrightarrow \\ 
    \varphi(s) &= y(s), s \in \partial B
\end{align*}
$$
Doing this also simplify choosing a boundary condition for $K(t,x)$:

$$
\begin{align*}
y(s) &= \int_{\partial B} y(x) K(s,x) dx \Leftarrow \\
K(s,x) &= \delta(s-x), s \in \partial B
\end{align*}
$$